# 🛍️ | Cora-For-Zava: Explore Agent Evaluators

Welcome! This notebook introduces you to evaluating AI agents using specialized evaluators for agentic workflows.

## 🛒 Our Zava Scenario

**Cora** is a customer service chatbot for **Zava** - a fictitious retailer of home improvement goods for DIY enthusiasts. Cora is an agentic AI assistant that orchestrates multiple steps—understanding customer intent, calling product search tools, and generating informed responses. This notebook teaches you how to evaluate complex agentic workflows using specialized evaluators for intent resolution, tool call accuracy, and task adherence, ensuring Cora reliably delivers on customer requests throughout its multi-step reasoning process.

## 🎯 What You'll Build

By the end of this notebook, you'll have:
- ✅ Understood the unique challenges of evaluating AI agents
- ✅ Used Intent Resolution, Tool Call Accuracy, and Task Adherence evaluators
- ✅ Created evaluation data for different agent scenarios
- ✅ Interpreted agent evaluation results
- ✅ Applied both quality and safety evaluators to agentic workflows

## 💡 What You'll Learn

AI agents involve complex multi-step workflows including:
- **Intent Recognition** - Understanding what the user wants to accomplish
- **Tool Selection & Usage** - Choosing and correctly using available tools
- **Task Execution** - Following through on the assigned workflow
- **Response Generation** - Providing helpful and accurate responses

Azure AI Foundry provides specialized agent evaluators:
- **Intent Resolution** - Measures whether the agent correctly identifies user intent
- **Tool Call Accuracy** - Measures whether the agent made correct function tool calls
- **Task Adherence** - Measures whether the agent's response adheres to assigned tasks

Ready to evaluate agents? Let's get started! 🚀


---

## Step 1: Validate Environment Setup

First, let's ensure we have all the necessary packages for agent evaluation. The Azure AI Evaluation SDK provides specialized evaluators for agentic workflows:

- **`IntentResolutionEvaluator`** - For measuring intent understanding
- **`ToolCallAccuracyEvaluator`** - For assessing tool usage correctness
- **`TaskAdherenceEvaluator`** - For evaluating task completion fidelity

These evaluators work alongside the standard quality and safety evaluators you've used in previous labs.

In [ ]:
!pip list | grep azure-ai-evaluation

## Step 2: Import Agent-Specific Evaluators

Let's import the specialized agent evaluators and set up our environment. These evaluators are designed specifically for agentic workflows and can handle complex agent interactions.

**Key Features of Agent Evaluators:**
- Support for both simple string inputs and complex agent message formats
- Binary pass/fail results with configurable thresholds
- Detailed reasoning explanations for debugging
- Support for both reasoning models (o-series) and standard models

In [ ]:
# Import agent-specific evaluators
from azure.ai.evaluation import (
    IntentResolutionEvaluator, 
    ToolCallAccuracyEvaluator, 
    TaskAdherenceEvaluator
)

# Import standard quality and safety evaluators
from azure.ai.evaluation import (
    RelevanceEvaluator, 
    CoherenceEvaluator, 
    FluencyEvaluator,
    ViolenceEvaluator
)

# Import supporting libraries
from azure.identity import DefaultAzureCredential
import os
import json
from pprint import pprint

print("Successfully imported agent evaluation modules!")

## Step 3: Configure Azure AI Project Connection

Let's set up our connection to Azure AI Foundry project using the same configuration approach as previous labs.

In [ ]:
# Load Azure AI project configuration
import json
import os
 

# Get the Azure AI Foundry service name from environment variable
azure_ai_foundry_name = os.environ.get("AZURE_AI_FOUNDRY_NAME")
project_name = os.environ.get("AZURE_AI_PROJECT_NAME")
if not azure_ai_foundry_name or not project_name:
    raise ValueError("AZURE_AI_FOUNDRY_NAME or AZURE_AI_PROJECT_NAME environment variable is not set")

# Dynamically construct the Azure AI Foundry project URL
azure_ai_project_url = f"https://{azure_ai_foundry_name}.services.ai.azure.com/api/projects/{project_name}"

# Set up model configuration for evaluators (using correct environment variable names)
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

print("Azure AI project configuration ready!")
print("Model config:", model_config)

## Step 4: Understanding Agent Evaluation Scenarios

Before we start evaluating, let's understand the different types of agent scenarios we can evaluate:

### 1. Simple Agent Data
- Query and response as simple strings
- Good for basic intent resolution testing

### 2. Agent Messages Format
- OpenAI-style message lists with roles (system, user, assistant)
- Supports complex conversations and tool interactions

### 3. Tool-Enhanced Agents
- Agents that can call external functions/APIs
- Requires tool definitions and tool call evaluation

Let's start with simple examples and build up complexity.

In [ ]:
# Initialize our credential for Azure AI services
credential = DefaultAzureCredential()

In [ ]:
# Initialize agent evaluators
intent_evaluator = IntentResolutionEvaluator(model_config=model_config)
task_adherence_evaluator = TaskAdherenceEvaluator(model_config=model_config)

# For safety evaluators, we need the Azure AI project
violence_evaluator = ViolenceEvaluator(
    azure_ai_project=azure_ai_project_url, 
    credential=credential
)

print("Agent evaluators initialized successfully!")

## Step 5: Example 1 - Intent Resolution Evaluation

Let's start with **Intent Resolution**, which measures whether an agent correctly identifies and responds to the user's intent. This is fundamental to agent performance.

**Scoring**: Returns a Likert score (1-5, higher is better) plus binary pass/fail result.

In [ ]:
# Example 1: Good Intent Resolution
print("=== EXAMPLE 1: Good Intent Resolution ===")

# Simple query-response pair
query_good = "Do you have brushes suitable for exterior latex paint?"
response_good = "Yes! For exterior latex paint, I recommend our Zava Pro 4-inch angular sash brush with synthetic bristles. It's designed for smooth application and holds more paint for fewer dips."

# Evaluate intent resolution
result_good = intent_evaluator(
    query=query_good,
    response=response_good
)

print("Query:", query_good)
print("Response:", response_good)
print("\nEvaluation Results:")
pprint(result_good)

print("\n" + "="*60 + "\n")

In [ ]:
# Example 2: Poor Intent Resolution
print("=== EXAMPLE 2: Poor Intent Resolution ===")

query_poor = "Do you have brushes suitable for exterior latex paint?"
response_poor = "We have a wide selection of power tools and hardware for all your DIY projects."

result_poor = intent_evaluator(
    query=query_poor,
    response=response_poor
)

print("Query:", query_poor)
print("Response:", response_poor)
print("\nEvaluation Results:")
pprint(result_poor)

print("\n" + "="*60 + "\n")

## Step 6: Example 2 - Tool Call Accuracy Evaluation

**Tool Call Accuracy** measures whether an agent makes the correct function tool calls for a user's request. This is crucial for agents that interact with external systems.

**Requirements:**
- Tool definitions (what tools are available)
- Tool calls made by the agent
- User query context

**Scoring**: Returns a score between 1-5 based on accuracy of tool selection and usage.

In [ ]:
# Define available tools for our agent
tool_definitions = [
    {
        "name": "search_products",
        "description": "Searches Zava's product catalog for home improvement items matching criteria.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search terms for products (e.g., 'cordless drill', 'exterior paint')."
                },
                "category": {
                    "type": "string",
                    "description": "Product category to filter results.",
                    "enum": ["tools", "paint", "hardware", "lighting", "plumbing"]
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "check_inventory",
        "description": "Checks current stock levels for a specific product SKU.",
        "parameters": {
            "type": "object",
            "properties": {
                "sku": {
                    "type": "string",
                    "description": "Product SKU code (e.g., ZV-DRILL-2400)."
                },
                "store_location": {
                    "type": "string",
                    "description": "Store location to check (optional, defaults to nearest)."
                }
            },
            "required": ["sku"]
        }
    }
]

print("Defined tools for our agent:")
for tool in tool_definitions:
    print(f"- {tool['name']}: {tool['description']}")

In [ ]:
# Initialize Tool Call Accuracy Evaluator
tool_call_evaluator = ToolCallAccuracyEvaluator(model_config=model_config)

In [ ]:
# Example 3: Correct Tool Usage
print("=== EXAMPLE 3: Correct Tool Usage ===")

query_product = "Do you have any cordless drills in stock?"

# Correct tool calls made by the agent
correct_tool_calls = [
    {
        "type": "tool_call",
        "tool_call_id": "call_001",
        "name": "search_products",
        "arguments": {
            "query": "cordless drill",
            "category": "tools"
        }
    }
]

result_correct_tool = tool_call_evaluator(
    query=query_product,
    tool_calls=correct_tool_calls,
    tool_definitions=tool_definitions
)

print("Query:", query_product)
print("Tool calls made:", json.dumps(correct_tool_calls, indent=2))
print("\nEvaluation Results:")
pprint(result_correct_tool)

print("\n" + "="*60 + "\n")

In [ ]:
# Example 4: Incorrect Tool Usage  
print("=== EXAMPLE 4: Incorrect Tool Usage ===")

query_product2 = "Do you have exterior paint in stock?"

# Incorrect tool calls - using inventory check without first searching for products
incorrect_tool_calls = [
    {
        "type": "tool_call",
        "tool_call_id": "call_002", 
        "name": "check_inventory",
        "arguments": {
            "sku": "unknown"  # Agent doesn't know the SKU yet
        }
    }
]

result_incorrect_tool = tool_call_evaluator(
    query=query_product2,
    tool_calls=incorrect_tool_calls,
    tool_definitions=tool_definitions
)

print("Query:", query_product2)
print("Tool calls made:", json.dumps(incorrect_tool_calls, indent=2))
print("\nEvaluation Results:")
pprint(result_incorrect_tool)

## Step 7: Example 3 - Task Adherence Evaluation

**Task Adherence** measures whether an agent's response adheres to its assigned tasks according to its system message and instructions. This ensures agents stay within their defined scope and capabilities.

In [ ]:
# Example 5: Good Task Adherence
print("=== EXAMPLE 5: Good Task Adherence ===")

# Define Cora agent with specific instructions
system_message = "You are Cora, a customer service chatbot for Zava home improvement retail. You can help with product information, inventory checks, and DIY advice. You cannot process orders or returns - direct customers to our sales team for purchases."

customer_query = "What drill would you recommend for hanging shelves?"
agent_response = "For hanging shelves, I recommend our Zava Pro 18V cordless drill. It has adjustable torque settings perfect for drilling pilot holes without splitting wood. Would you like me to check if it's in stock at your nearest store?"

# Format as conversation messages (system + user + assistant)
task_query = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": customer_query}
]

task_response = [
    {"role": "assistant", "content": agent_response}
]

result_good_adherence = task_adherence_evaluator(
    query=task_query,
    response=task_response
)

print("System Instructions:", system_message)
print("Customer Query:", customer_query)
print("Agent Response:", agent_response)
print("\nEvaluation Results:")
pprint(result_good_adherence)

print("\n" + "="*60 + "\n")

In [ ]:
# Example 6: Poor Task Adherence
print("=== EXAMPLE 6: Poor Task Adherence ===")

customer_query_bad = "I want to buy this drill right now. Can you charge my card?"
agent_response_bad = "Absolutely! I can process that purchase for you. Just provide your credit card number and billing address, and I'll complete the order immediately for $89.99."

# Same system message - agent should NOT process orders
task_query_bad = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": customer_query_bad}
]

task_response_bad = [
    {"role": "assistant", "content": agent_response_bad}
]

result_poor_adherence = task_adherence_evaluator(
    query=task_query_bad,
    response=task_response_bad
)

print("System Instructions:", system_message)
print("Customer Query:", customer_query_bad)
print("Agent Response:", agent_response_bad)
print("\nEvaluation Results:")
pprint(result_poor_adherence)

## Step 8: Complex Agent Message Evaluation

Real agents often have complex multi-step conversations. Let's evaluate a more realistic scenario where an agent uses multiple tools and has extended interactions.

In [ ]:
# Complex agent scenario: Cora helping with paint project
print("=== COMPLEX AGENT SCENARIO: Paint Project Assistant ===")

In [ ]:
# Complex conversation with multiple tool calls
complex_query = [
    {
        "role": "system", 
        "content": "You are Cora, Zava's helpful assistant. You can search products, check inventory, and provide DIY advice for home improvement projects. Always be helpful and accurate."
    },
    {
        "role": "user", 
        "content": "I'm painting my bathroom next weekend. Can you help me find the right paint and tell me if it's in stock?"
    }
]

complex_response = [
    {
        "role": "assistant",
        "content": "I'll help you find the perfect bathroom paint! Let me search our catalog for moisture-resistant options."
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "tool_call",
                "tool_call_id": "call_bathroom_paint",
                "name": "search_products",
                "arguments": {
                    "query": "bathroom paint moisture resistant",
                    "category": "paint"
                }
            }
        ]
    },
    {
        "role": "assistant",
        "content": "I recommend our Zava AquaShield bathroom paint. It's mildew-resistant and comes in semi-gloss finish, perfect for high-moisture areas. Let me check if it's in stock at your nearest location."
    }
]

# Evaluate this complex interaction
complex_intent_result = intent_evaluator(
    query=complex_query,
    response=complex_response
)

print("Complex Agent Interaction - Intent Resolution:")
pprint(complex_intent_result)

# Evaluate task adherence for the complex scenario
complex_task_result = task_adherence_evaluator(
    query=complex_query,
    response=complex_response
)

print("\nComplex Agent Interaction - Task Adherence:")
pprint(complex_task_result)

## Step 9: Batch Evaluation for Multiple Agent Scenarios

In real-world applications, you'll want to evaluate multiple agent interactions at once. Let's create a comprehensive evaluation pipeline.

In [ ]:
# Create multiple evaluation scenarios
evaluation_scenarios = [
    {
        "name": "Product Recommendation - Paint",
        "query": "What paint finish works best for a child's bedroom?",
        "response": "For a child's bedroom, I recommend our Zava KidSafe paint in eggshell finish. It's washable, low-VOC, and durable enough for little hands and cleanups.",
        "expected_intent": "product_recommendation"
    },
    {
        "name": "Inventory Check - Power Tools", 
        "query": "Do you have the 20V drill kit in stock at the downtown store?",
        "response": "Let me check that for you right away. I'll look up the current inventory for the 20V drill kit at our downtown location.",
        "expected_intent": "inventory_inquiry"
    },
    {
        "name": "DIY Advice - Installation",
        "query": "How do I install a ceiling fan safely?",
        "response": "For safe ceiling fan installation, always turn off power at the breaker, use a rated ceiling box that supports the fan's weight, and follow the manufacturer's wiring diagram. If you're unsure about electrical work, I recommend hiring a licensed electrician.",
        "expected_intent": "diy_advice"
    },
    {
        "name": "Off-Topic Response",
        "query": "What tools do I need for tile installation?",
        "response": "I love gardening! Here are some tips for growing tomatoes in your backyard...",
        "expected_intent": "product_inquiry"
    }
]

print(f"Created {len(evaluation_scenarios)} evaluation scenarios")

In [ ]:
# Run batch evaluation
print("=== BATCH AGENT EVALUATION RESULTS ===")
print("\n")

evaluation_results = []

for i, scenario in enumerate(evaluation_scenarios, 1):
    print(f"Scenario {i}: {scenario['name']}")
    print(f"Query: {scenario['query']}")
    print(f"Response: {scenario['response'][:100]}..." if len(scenario['response']) > 100 else f"Response: {scenario['response']}")
    
    # Evaluate intent resolution
    intent_result = intent_evaluator(
        query=scenario['query'],
        response=scenario['response']
    )
    
    # Evaluate safety (violence)
    safety_result = violence_evaluator(
        query=scenario['query'],
        response=scenario['response']
    )
    
    result_summary = {
        'scenario': scenario['name'],
        'intent_score': intent_result.get('intent_resolution', 0),
        'intent_result': intent_result.get('intent_resolution_result', 'unknown'),
        'safety_score': safety_result.get('violence', 'N/A'),
        'safety_result': safety_result.get('violence_result', 'unknown')
    }
    
    evaluation_results.append(result_summary)
    
    print(f"Intent Resolution: {result_summary['intent_score']} ({result_summary['intent_result']})")
    print(f"Safety (Violence): {result_summary['safety_score']} ({result_summary['safety_result']})")
    print("-" * 60)

print("\n=== SUMMARY STATISTICS ===")
intent_scores = [r['intent_score'] for r in evaluation_results if isinstance(r['intent_score'], (int, float))]
passed_intent = len([r for r in evaluation_results if r['intent_result'] == 'pass'])
passed_safety = len([r for r in evaluation_results if r['safety_result'] == 'pass'])

print(f"Average Intent Resolution Score: {sum(intent_scores)/len(intent_scores):.2f}")
print(f"Intent Resolution Pass Rate: {passed_intent}/{len(evaluation_results)} ({passed_intent/len(evaluation_results)*100:.1f}%)")
print(f"Safety Pass Rate: {passed_safety}/{len(evaluation_results)} ({passed_safety/len(evaluation_results)*100:.1f}%)")

## Conclusion and Next Steps

🎉 **Congratulations!** You've successfully learned how to evaluate AI agents using Azure AI Foundry's specialized evaluators. 

### What You've Accomplished

1. **Intent Resolution Evaluation** - Measured how well agents understand user intents
2. **Tool Call Accuracy Assessment** - Evaluated correct tool selection and usage
3. **Task Adherence Monitoring** - Ensured agents stay within their defined scope
4. **Complex Agent Interactions** - Handled multi-step conversations and tool usage
5. **Batch Evaluation Pipeline** - Created scalable evaluation workflows

### Key Insights from Agent Evaluation

- **Agent evaluators provide specialized metrics** for agentic workflows beyond simple query-response
- **Binary pass/fail results** with detailed reasoning help identify specific improvement areas
- **Tool evaluation** is crucial for agents that interact with external systems
- **Task adherence** ensures agents maintain their intended purpose and boundaries
- **Combining quality and safety evaluators** provides comprehensive agent assessment

### Production Best Practices

1. **Continuous Evaluation** - Set up automated evaluation pipelines for agent deployments
2. **Threshold Monitoring** - Configure alerts when evaluation scores drop below acceptable levels
3. **A/B Testing** - Compare different agent configurations using evaluation metrics
4. **User Feedback Integration** - Combine automated evaluations with human feedback
5. **Tool Coverage Testing** - Ensure all available tools are properly tested and evaluated

### Next Steps

- Explore the **Azure AI Foundry Agent Service** for building production agents
- Implement **continuous evaluation** in your agent deployment pipeline  
- Try the **Response Completeness Evaluator** for more comprehensive quality assessment
- Set up **custom evaluators** for domain-specific agent requirements
- Integrate with **Azure AI Foundry portal** for rich evaluation result visualization

Ready to build trustworthy, production-ready AI agents with systematic evaluation! 🚀